## Amazon Web Scraper

In [1]:
#Installing Libraries

import csv
from bs4 import BeautifulSoup

from msedge.selenium_tools import Edge, EdgeOptions
from selenium import webdriver

In [2]:
options = EdgeOptions()
options.use_chromium = True
driver = webdriver.Edge(executable_path = r'C:\Program Files (x86)\Microsoft\Edge Webdriver\msedgedriver.exe')

In [3]:
url = "https://www.amazon.in"

driver.get(url)

In [55]:
def get_url(search_term):
    template = 'https://www.amazon.in/s?k={}&crid=AWC66IMNQMNW&sprefix={}%2Caps%2C238&ref=nb_sb_noss'
    search_term = search_term.replace(' ', '+')
    return f'https://www.amazon.in/s?k={search_term}&crid=AWC66IMNQMNW&sprefix={search_term}%2Caps%2C238&ref=nb_sb_noss'

In [57]:
url = get_url('wireless headphones')
print(url)

https://www.amazon.in/s?k=wireless+headphones&crid=AWC66IMNQMNW&sprefix=wireless+headphones%2Caps%2C238&ref=nb_sb_noss


In [6]:
driver.get(url)

## Extracting page source

In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [9]:
print(len(results))

22


## Prototype the record

In [10]:
item = results[0]

In [11]:
atag = item.h2.a

In [16]:
description = atag.text.strip()

In [20]:
url = 'https://www.amazon.in'+ atag.get('href')

In [22]:
print(url)

https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A02041603KLJT5SRJTC2I&url=%2FBoat-Rockerz-550-Headphone-Aesthetics%2Fdp%2FB0856HY85J%2Fref%3Dsr_1_1_sspa%3Fcrid%3DAWC66IMNQMNW%26keywords%3Dwireless%2Bheadphones%26qid%3D1641733878%26sprefix%3Dwireless%2Bheadphones%252Caps%252C238%26sr%3D8-1-spons%26psc%3D1&qualifier=1641733878&id=7434389333570517&widgetName=sp_atf


In [32]:
parent_price = item.find('span', 'a-price')

In [34]:
price = parent_price.find('span', 'a-offscreen').text

In [37]:
rating = item.i.text

In [42]:
review_count = item.find('span', 'a-size-base').text

## Generalizing the pattern

In [44]:
def extract_info(item):
    
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in'+ atag.get('href')
    
    try:
        parent_price = item.find('span', 'a-price')
        price = parent_price.find('span', 'a-offscreen').text
    except AttributeError:
        price = 'Not Available'
    
    try:
        rating = item.i.text
        review_count = item.find('span', 'a-size-base').text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

In [45]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_info(item)
    if record:
        records.append(record)

In [47]:
records[0]

('boAt Rockerz 550 Bluetooth Wireless Over Ear Headphone with Mic (Black)',
 '₹1,899',
 '4.1 out of 5 stars',
 '34,840',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A02041603KLJT5SRJTC2I&url=%2FBoat-Rockerz-550-Headphone-Aesthetics%2Fdp%2FB0856HY85J%2Fref%3Dsr_1_1_sspa%3Fcrid%3DAWC66IMNQMNW%26keywords%3Dwireless%2Bheadphones%26qid%3D1641733878%26sprefix%3Dwireless%2Bheadphones%252Caps%252C238%26sr%3D8-1-spons%26psc%3D1&qualifier=1641733878&id=7434389333570517&widgetName=sp_atf')

In [50]:
for row in records:
    print(row[1])

₹1,899
₹2,419
₹999
₹990
₹999
₹1,299
₹999
₹1,899
₹599
₹798
₹16,990
₹999
₹749
₹1,599
₹999
₹999
₹1,299
₹799
₹1,499
₹1,999
₹1,299
₹999


## Getting to the next page

In [58]:
def get_url(search_term):
    template = 'https://www.amazon.in/s?k=wireless+headphones&crid=AWC66IMNQMNW&sprefix=wireless+headphones%2Caps%2C238&ref=nb_sb_noss'
    search_term = search_term.replace(' ', '+')
    template.format(search_term)
    url += '&page={}'
    return url

## Putting everything together

In [64]:
import csv
from bs4 import BeautifulSoup

from msedge.selenium_tools import Edge, EdgeOptions
from selenium import webdriver


def get_url(search_term):
    
    template = 'https://www.amazon.in/s?k={}&crid=AWC66IMNQMNW&sprefix={}%2Caps%2C238&ref=nb_sb_noss'
    search_term = search_term.replace(' ', '+')
    url =  f'https://www.amazon.in/s?k={search_term}&crid=AWC66IMNQMNW&sprefix={search_term}%2Caps%2C238&ref=nb_sb_noss'
    url += '&page={}'
    
    return url


def extract_info(item):
    
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.in'+ atag.get('href')
    
    try:
        parent_price = item.find('span', 'a-price')
        price = parent_price.find('span', 'a-offscreen').text
    except AttributeError:
        price = 'Not Available'
    
    try:
        rating = item.i.text
        review_count = item.find('span', 'a-size-base').text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    
    options = EdgeOptions()
    options.use_chromium = True
    driver = webdriver.Edge(executable_path = r'C:\Program Files (x86)\Microsoft\Edge Webdriver\msedgedriver.exe')
    
    records = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_info(item)
            if record:
                records.append(record)
                
    driver.close()
    
    with open('scraped_data.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Review Count', 'URL'])
        writer.writerows(records)

In [65]:
main('wireless earphones')